### Read data file from online JSON file

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import datetime as dt
import matplotlib.dates as mdates
plt.rcParams['figure.dpi'] = 100

# set conditions
countryCode_a = "FRA"
countryCode_b = "DEU"
key = "new_cases_smoothed"
blue = "#1f77b4"
orange = "#ff7f0e"
diffDayMax = +100
diffDayMin = -100

# read JSON file from URL
data = requests.get("https://covid.ourworldindata.org/data/owid-covid-data.json").json()
#f = open('owid-covid-data.json', 'r')
#data = json.load(f)
country_a = data[countryCode_a]["data"]
country_b = data[countryCode_b]["data"]
name_a = data[countryCode_a]["location"]
name_b = data[countryCode_b]["location"]

# show sample key list
print("Key list:")
for sample_key in country_a[-2]:
    print(sample_key)

### Show time-series data 

In [ ]:
dates_a = []
dates_b = []
value_a = []
value_b = []

for datedata in country_a: # read data of country_a
    t = datedata.get(key, "nan") # if there is no key, return nan

    if(t != "nan"):
        value_a.append(float(t))
        dates_a.append(datedata.get("date"))

for datedata in country_b: # read data of country_b
    t = datedata.get(key, "nan") # if there is no key, return nan

    if(t != "nan"):
        value_b.append(float(t))
        dates_b.append(datedata.get("date"))

# convert into datatime format
dates_a = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in dates_a]
dates_b = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in dates_b]

# visualize
fig, ax1 = plt.subplots()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120))

ax2 = ax1.twinx()
ax1.set_title(key)
ax1.plot(dates_a, value_a, blue)
ax2.plot(dates_b, value_b, orange)
plt.gcf().autofmt_xdate()
ax1.set_ylabel(name_a, color=blue)
ax2.set_ylabel(name_b, color=orange)
plt.show()

### Show correlation

In [ ]:
# compute mean and standard deviation 
va_std  = np.std(value_a)
va_mean = np.mean(value_a)
vb_std  = np.std(value_b)
vb_mean = np.mean(value_b)

numDates_a = len(dates_a)
numDates_b = len(dates_b)
maxDiffDay_a = (dates_a[-1] - dates_a[0]).days + 1
maxDiffDay_b = (dates_b[-1] - dates_b[0]).days + 1

correlSize = maxDiffDay_a + maxDiffDay_b -1
diffs = np.arange(- (maxDiffDay_b - 1), maxDiffDay_a)
correlations = np.zeros([correlSize])
counts = np.zeros([correlSize])

# compute correlation
for a_day in range(numDates_a):
    for b_day in range(numDates_b):
        diffDay = (dates_b[b_day] - dates_a[a_day]).days + (maxDiffDay_b - 1)

        correlations[diffDay] += (value_a[a_day] - va_mean)*(value_b[b_day] - vb_mean)
        counts[diffDay] += 1

correlations /= va_std*vb_std*counts

# get information of maximum correlation
diffs = np.array(diffs)
correlations = np.array(correlations)
cliped_diffs = diffs[(diffs > diffDayMin) & (diffs < diffDayMax)]
cliped_correlations = correlations[(diffs > diffDayMin) & (diffs < diffDayMax)]
max_day    = cliped_diffs[cliped_correlations.argmax()]
max_correl = cliped_correlations[cliped_correlations.argmax()]

print("----- Max correlation:", max_day, max_correl)

# visualize
fig, ax = plt.subplots()
ax.set_title(key)
ax.plot(diffs, correlations)
ax.plot(max_day, max_correl, "ro")
ax.set_xlabel('diff_day')
ax.set_ylabel('correlation')
ax.set_xlim([diffDayMin, diffDayMax])
plt.show()